In [7]:
import os 
from dotenv import load_dotenv
load_dotenv('../../.env')

model=os.environ.get('model')
print(model)

os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get('AZURE_API_KEY')
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get('AZURE_API_KEY')
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ.get('AZURE_API_VERSION')
os.environ["AUTOGEN_USE_DOCKER"] = "0"

azure/gpt-4


In [13]:
import autogen
from typing import Dict, List, Tuple, Optional, Literal, Union


In [32]:
import os
from autogen import AzureOpenAIAgent

ImportError: cannot import name 'AzureOpenAIAgent' from 'autogen' (C:\Temp\openai\gen-ai-sd\crewai\.venv\lib\site-packages\autogen\__init__.py)

In [14]:
# Configuration for Azure OpenAI
config_list = [
    {
        "model": "gpt-4",  # Or your specific Azure OpenAI deployment name
        "api_type": "azure",
        "api_key": os.environ.get('AZURE_API_KEY'),
        "api_base": os.environ.get('AZURE_API_KEY'),  # Your Azure OpenAI endpoint
        "api_version": os.environ.get('AZURE_API_VERSION')  # Update with your API version
    }
]

In [15]:
# Configure termination message for conversations
termination_msg = lambda x: isinstance(x, str) and "TERMINATE" in x

In [16]:
# Define user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "workspace"},
    system_message="I am a user proxy that represents the customer."
)

In [17]:
# Create agents for travel domain
flight_agent = autogen.AssistantAgent(
    name="flight_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a flight booking specialist. You help customers with flight booking related queries.
    End your response with TERMINATE when the task is complete."""
)

train_agent = autogen.AssistantAgent(
    name="train_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a train booking specialist. You help customers with train booking related queries.
    End your response with TERMINATE when the task is complete."""
)

travel_assistant_agent = autogen.AssistantAgent(
    name="travel_assistant_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a general travel assistant. You help customers with general travel related queries 
    that are not specifically about flights or trains.
    End your response with TERMINATE when the task is complete."""
)

# Create agents for finance domain
stock_agent = autogen.AssistantAgent(
    name="stock_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a stock market specialist. You help customers with stock market related queries.
    End your response with TERMINATE when the task is complete."""
)

mutual_fund_agent = autogen.AssistantAgent(
    name="mutual_fund_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a mutual fund specialist. You help customers with mutual fund related queries.
    End your response with TERMINATE when the task is complete."""
)

finance_assistant_agent = autogen.AssistantAgent(
    name="finance_assistant_agent",
    llm_config={"config_list": config_list},
    system_message="""You are a general finance assistant. You help customers with general finance related queries 
    that are not specifically about stocks or mutual funds.
    End your response with TERMINATE when the task is complete."""
)

In [18]:
# Create travel supervisor agent
class TravelSupervisorAgent(autogen.AssistantAgent):
    def __init__(self):
        super().__init__(
            name="travel_supervisor",
            llm_config={"config_list": config_list},
            system_message="""You are a travel domain supervisor. 
            Your job is to analyze customer queries and route them to the appropriate travel agent:
            - flight_agent: For flight booking, flight status, or any flight related queries
            - train_agent: For train booking, train status, or any train related queries
            - travel_assistant_agent: For all other travel related queries
            
            Determine the most appropriate agent and explain your routing decision."""
        )
        self.register_reply(
            [flight_agent, train_agent, travel_assistant_agent, user_proxy],
            TravelSupervisorAgent.route_query
        )
    
    @staticmethod
    def route_query(self, messages: List[Dict], sender, config):
        query = messages[-1]["content"]
        # Simple keyword-based routing logic
        if any(keyword in query.lower() for keyword in ["flight", "airplane", "airport", "airline"]):
            target_agent = flight_agent
            routing_explanation = "Routing to flight_agent as the query appears to be flight-related."
        elif any(keyword in query.lower() for keyword in ["train", "railway", "station", "rail"]):
            target_agent = train_agent
            routing_explanation = "Routing to train_agent as the query appears to be train-related."
        else:
            target_agent = travel_assistant_agent
            routing_explanation = "Routing to travel_assistant_agent as the query is travel-related but not specifically about flights or trains."
        
        response = f"{routing_explanation}\n\n@{target_agent.name} Please address the following travel query: {query}"
        return {"content": response}

# Create finance supervisor agent
class FinanceSupervisorAgent(autogen.AssistantAgent):
    def __init__(self):
        super().__init__(
            name="finance_supervisor",
            llm_config={"config_list": config_list},
            system_message="""You are a finance domain supervisor. 
            Your job is to analyze customer queries and route them to the appropriate finance agent:
            - stock_agent: For stock market, trading, shares, or any stock related queries
            - mutual_fund_agent: For mutual funds, index funds, ETFs, or any fund related queries
            - finance_assistant_agent: For all other finance related queries
            
            Determine the most appropriate agent and explain your routing decision."""
        )
        self.register_reply(
            [stock_agent, mutual_fund_agent, finance_assistant_agent, user_proxy],
            FinanceSupervisorAgent.route_query
        )
    
    @staticmethod
    def route_query(self, messages: List[Dict], sender, config):
        query = messages[-1]["content"]
        # Simple keyword-based routing logic
        if any(keyword in query.lower() for keyword in ["stock", "share", "trading", "market", "investor", "equity"]):
            target_agent = stock_agent
            routing_explanation = "Routing to stock_agent as the query appears to be stock market-related."
        elif any(keyword in query.lower() for keyword in ["mutual fund", "etf", "index fund", "fund", "nav", "sip"]):
            target_agent = mutual_fund_agent
            routing_explanation = "Routing to mutual_fund_agent as the query appears to be fund-related."
        else:
            target_agent = finance_assistant_agent
            routing_explanation = "Routing to finance_assistant_agent as the query is finance-related but not specifically about stocks or mutual funds."
        
        response = f"{routing_explanation}\n\n@{target_agent.name} Please address the following finance query: {query}"
        return {"content": response}

In [19]:
# Create main supervisor agent
class MainSupervisorAgent(autogen.AssistantAgent):
    def __init__(self):
        super().__init__(
            name="main_supervisor",
            llm_config={"config_list": config_list},
            system_message="""You are the main supervisor agent. 
            Your job is to analyze customer queries and route them to the appropriate domain supervisor:
            - travel_supervisor: For travel related queries including flights, trains, hotels, vacations, etc.
            - finance_supervisor: For finance related queries including stocks, mutual funds, investments, etc.
            
            Determine the most appropriate supervisor and explain your routing decision."""
        )
        self.register_reply(
            [travel_supervisor, finance_supervisor, user_proxy],
            MainSupervisorAgent.route_query
        )
    
    @staticmethod
    def route_query(self, messages: List[Dict], sender, config):
        query = messages[-1]["content"]
        # Simple keyword-based routing logic
        travel_keywords = ["travel", "flight", "train", "hotel", "vacation", "booking", "trip", "journey", "tourist", "airport", "station"]
        finance_keywords = ["finance", "money", "investment", "stock", "fund", "mutual", "share", "trading", "market", "portfolio", "dividend", "financial"]
        
        travel_score = sum(1 for keyword in travel_keywords if keyword in query.lower())
        finance_score = sum(1 for keyword in finance_keywords if keyword in query.lower())
        
        if travel_score > finance_score:
            target_supervisor = travel_supervisor
            routing_explanation = "Routing to travel_supervisor as the query appears to be travel-related."
        else:
            target_supervisor = finance_supervisor
            routing_explanation = "Routing to finance_supervisor as the query appears to be finance-related."
        
        response = f"{routing_explanation}\n\n@{target_supervisor.name} Please determine the appropriate agent for the following query: {query}"
        return {"content": response}

In [20]:
# Initialize supervisor agents
travel_supervisor = TravelSupervisorAgent()
finance_supervisor = FinanceSupervisorAgent()
main_supervisor = MainSupervisorAgent()

In [21]:
# Define the agent groupchat for each domain
travel_groupchat = autogen.GroupChat(
    agents=[user_proxy, travel_supervisor, flight_agent, train_agent, travel_assistant_agent],
    messages=[],
    max_round=10
)

finance_groupchat = autogen.GroupChat(
    agents=[user_proxy, finance_supervisor, stock_agent, mutual_fund_agent, finance_assistant_agent],
    messages=[],
    max_round=10
)

In [22]:
# Define the main groupchat
main_groupchat = autogen.GroupChat(
    agents=[user_proxy, main_supervisor, travel_supervisor, finance_supervisor],
    messages=[],
    max_round=10
)


In [29]:
# Initialize managers for each groupchat
travel_manager = autogen.GroupChatManager(groupchat=travel_groupchat, llm_config={"config_list": config_list}, is_termination_msg=termination_msg)
finance_manager = autogen.GroupChatManager(groupchat=finance_groupchat, llm_config={"config_list": config_list}, is_termination_msg=termination_msg)
main_manager = autogen.GroupChatManager(groupchat=main_groupchat, llm_config={"config_list": config_list}, is_termination_msg=termination_msg)


In [30]:
def process_customer_query(query: str):
    """Process a customer query through the hierarchical multi-agent system"""
    
    print(f"Customer Query: {query}")
    print("-" * 50)
    
    # Start the conversation with the main supervisor
    user_proxy.initiate_chat(main_manager, message=query)

    # The conversation flow:
    # 1. main_supervisor determines if query is travel or finance related
    # 2. routes to appropriate domain supervisor
    # 3. domain supervisor routes to specific agent
    # 4. specific agent addresses the query

In [31]:
# Example usage
if __name__ == "__main__":
    # Example travel query
    process_customer_query("I need to book a flight from New York to London next week")
    
    # Example finance query
    process_customer_query("What are the best mutual funds to invest in for long-term growth?")

Customer Query: I need to book a flight from New York to London next week
--------------------------------------------------
user_proxy (to chat_manager):

I need to book a flight from New York to London next week

--------------------------------------------------------------------------------


TypeError: Completions.create() got an unexpected keyword argument 'api_base'